<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/yykim/LDA_%EC%A7%81%EC%A0%91_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LDA 직접 구현

In [1]:
d1 = 'Cute kitty'
d2 = 'Eat rice or cake'
d3 = 'Kitty and hamster'
d4 = 'Eat bread'
d5 = 'Rice, bread and cake'
d6 = 'Cute hamster eats bread and cake'

docs = [d1, d2, d3, d4, d5,d6]
docs = [i.replace(",","") for i in docs]
docs = [i.replace("!","") for i in docs]
docs = [i.replace(".","") for i in docs]
docs = [i.replace("\'","") for i in docs]
docs = [i.replace("\"","") for i in docs]
docs = [i.replace(":","") for i in docs]
docs = [i.replace(";","") for i in docs]
docs = [i.replace("and","") for i in docs]
docs = [i.replace("or","") for i in docs]


##Class

In [2]:
import numpy as np
import random
import copy

class LDA:
    def __init__(self, doc_ls, topic_num, alpha = 0.1, beta = 0.001, max_iter = 50):
        self.alpha = alpha
        self.beta = beta
        self.k = topic_num
        self.max_iter = max_iter
        self.total_tokens = []
        self.unique_total_tokens = []
        self.doc_ls = doc_ls
    
    def _assign_topic_randomly(self):
        tokenized_docs = [doc.split(" ") for doc in self.doc_ls] #문서별로 토큰화

        self.total_tokens = [] #문서 구분 없이 토큰화, 유니크 토큰 뽑기
        for sentence in self.doc_ls:
            self.total_tokens.extend(sentence.split(" "))
        self.unique_total_tokens = list(np.unique(self.total_tokens))

        docs_tokens_topic = tokenized_docs.copy()
        for i, tokenized_doc in enumerate(docs_tokens_topic): #문서별로 토큰화된 토큰에 랜덤으로 토픽 부여
            docs_tokens_topic[i] = [ random.randint(0,100) % self.k for token in tokenized_doc]

        return docs_tokens_topic

    def _cal_topic_in_doc(self):        
        docs_tokens_topic = self._assign_topic_randomly()
        topic_in_doc = []
        for topic in range(self.k):
            tmp = []
            for doc_tokens_topic in docs_tokens_topic:
                    tmp.append(doc_tokens_topic.count(topic))
            topic_in_doc.append(tmp)
        return np.array(topic_in_doc) + self.alpha

    def _cal_words_in_topic(self):
        docs_tokens_topic = self._assign_topic_randomly()
        topic_token = []
        for doc in docs_tokens_topic:
            topic_token.extend(doc)

        topics_count_per_token=[]
        for u_token in self.unique_total_tokens:
            tmp = []
            topic_count = []
            for idx, token in enumerate(self.total_tokens):
                if token == u_token:
                    tmp.append(topic_token[idx])
            for topic in range(self.k):
                topic_count.append(tmp.count(topic))
            topics_count_per_token.append(topic_count)
        return np.array(topics_count_per_token).T + self.beta

    def fit(self):
        docs_tokens_topic = self._assign_topic_randomly()
        pre_docs_tokens_topic = docs_tokens_topic.copy()

        for iter in range(self.max_iter):
            for i, doc in enumerate([doc.split(" ") for doc in self.doc_ls]):
                for ix, token in enumerate(doc):
                    idx = self.unique_total_tokens.index(token) #토큰의 인덱스
                    which_topic = [] 
                    for topic in range(self.k):
                        if topic == docs_tokens_topic[i][ix]:
                            which_topic.append(((self._cal_topic_in_doc()[topic][i]-1)/(self._cal_topic_in_doc()[:,[i]].sum()-1))*((self._cal_words_in_topic()[topic][idx]-1)/(self._cal_words_in_topic()[topic].sum()-1)))
                        else:
                            which_topic.append((self._cal_topic_in_doc()[topic][i]/(self._cal_topic_in_doc()[:,[i]].sum()-1))*(self._cal_words_in_topic()[topic][idx]/(self._cal_words_in_topic()[topic].sum()-1)))
                    pre_docs_tokens_topic[i][ix] = np.argmax(np.array(which_topic))

            if docs_tokens_topic == pre_docs_tokens_topic:
                break    

            docs_tokens_topic = pre_docs_tokens_topic

    def predict(self):
        self._assign_topic_randomly()
        self._cal_topic_in_doc()
        words_in_tp = self._cal_words_in_topic()
        self.fit()
        for topic in range(self.k):
            np.argsort(words_in_tp[topic])[::-1][:4]
            top = np.argsort(words_in_tp)[topic][::-1][:4]
            tmp=[]
            for i in top:
                tmp.append((self.unique_total_tokens[i], (words_in_tp[topic][i]/(words_in_tp[topic].sum()-1))))
            print("Topic{}: {}".format(topic, tmp))
    

In [3]:
lda = LDA(doc_ls = docs, topic_num=2)

In [4]:
lda.predict()

Topic0: [('cake', 0.2725456361819999), ('', 0.2725456361819999), ('bread', 0.1817273635455454), ('hamster', 0.09090909090909093)]
Topic1: [('rice', 0.12495318936462367), ('kitty', 0.12495318936462367), ('hamster', 0.12495318936462367), ('bread', 0.12495318936462367)]
